In [1]:
!pip install spacy
!python -m spacy download de_core_news_sm

  Using cached spacy-3.4.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.4 MB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached srsly-2.4.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (459 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached thinc-8.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (813 kB)
  Using cached pydantic-1.9.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached typer-0.4.2-py3-none-any.whl (27 kB)
  Using cached spacy_legacy-3.0.10-py2.py3-none-any.whl (21 kB)
  Using cached preshed-3.0.7-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (128 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached murmurhash-1.0.8-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached spacy_loggers-1.0.3-py3-none-any.whl (9.3 kB)
  Using ca

In [2]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter
import spacy

nlp = spacy.load("de_core_news_sm")

2022-09-15 08:18:17.413854: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
def preprocessing(dataset):
    function = []
    non_function = []
    nonnoun = []
    nonnoun_verb = []
    nonverb = []
    nonverb_aux = []
    nonnoun_only = []
    listofpostypes = ['AUX', 'ADP', 'DET', 'CONJ', 'PRON']
    adj_nouns = ['ADJ', 'NOUN', 'PROPN']
    adj_nouns_verbs = ['ADJ', 'NOUN', 'PROPN', 'VERB']
    verbs = ['VERB']
    verbs_aux = ['VERB', 'AUX']
    nouns = ['NOUN', 'PROPN']

    for index, example in enumerate(dataset["Sentence"]):
        func_sent = [token.text for token in nlp(example) if token.pos_ in listofpostypes]
        nonfunc_sent = [token.text for token in nlp(example) if not token.pos_ in listofpostypes]
        nonnoun_sent = [token.text for token in nlp(example) if not token.pos_ in adj_nouns]
        nonnoun_verb_sent = [token.text for token in nlp(example) if not token.pos_ in adj_nouns_verbs]
        nonverb_sent = [token.text for token in nlp(example) if not token.pos_ in verbs]
        nonverb_aux_sent = [token.text for token in nlp(example) if not token.pos_ in verbs_aux]
        nonnoun_only_sent = [token.text for token in nlp(example) if not token.pos_ in nouns]
        function.append(' '.join(func_sent))
        non_function.append(' '.join(nonfunc_sent))
        nonnoun.append(' '.join(nonnoun_sent))
        nonnoun_verb.append(' '.join(nonnoun_verb_sent))
        nonverb.append(' '.join(nonverb_sent))
        nonverb_aux.append(' '.join(nonverb_aux_sent))
        nonnoun_only.append(' '.join(nonnoun_only_sent))
        #negative = ([negative + 1 for token in nlp(example) if token.lemma_ in neg])
    dataset["function_Sentence"] = function
    dataset["nonfunction_Sentence"] = non_function
    dataset["non_noun_adj"] = nonnoun
    dataset["non_noun_verb_adj"] = nonnoun_verb
    dataset["non_verb"] = nonverb
    dataset["non_verb_aux"] = nonverb_aux
    dataset["non_noun"] = nonnoun_only
    return dataset

In [5]:
#training_data = pd.read_csv("forum/label_train_preproc_drop.csv", index_col="Unnamed: 0")
#Labels:
'''datasets = ["01_random_split/label_test.csv", "03_non_smoking/nichtraucher_labels_v2.csv",
            "04_synthetic/output_labels.csv", "05_optifast/optifast_labels.csv", 
            "06_instagram_data/instagram_labels.csv", "07_VirtualCoachData/virtual_coach_labels.csv",
            "08_WoZ/WoZ_labels.csv"]'''

#sublabels
'''datasets = ["01_random_split/sublabel_train.csv", "01_random_split/sublabel_test.csv", "03_non_smoking/nichtraucher_sublabels_v2.csv",
            "04_synthetic/output_sublabels.csv", "05_optifast/optifast_sublabels.csv", 
            "06_instagram_data/instagram_sublabels.csv", "07_VirtualCoachData/virtual_coach_sublabels.csv",
            "08_WoZ/WoZ_sublabels.csv"]

names = ["train", "test", "nonsmoking", "synthetic", "optifast", "instagram", "virtualcoach", "WoZ"]'''

#valence
datasets = ["01_random_split/binary_train.csv", "01_random_split/binary_test.csv", "03_non_smoking/nichtraucher_valence_v2.csv",
            "05_optifast/optifast_valence.csv",  "04_synthetic/output_valences.csv",
            "07_VirtualCoachData/virtual_coach_valence.csv", "08_WoZ/WoZ_valence.csv"]

names = ["train", "test", "nonsmoking", "optifast", "synthetic", "virtualcoach", "WoZ"]

for index, i in enumerate(datasets):
    test_data = pd.read_csv("../MI_Data/Bert_Finetuning/" + i, index_col="Unnamed: 0")
    test_data_prep = preprocessing(test_data)
    nan_value = float("NaN")
    test_data_prep.replace("", nan_value, inplace=True)
    test_data_prep.dropna(how='any', inplace=True)
    set_name = str(names[index])
    #test_data_prep.to_csv("forum/labels/"+set_name+"_label_preproc.csv")
    #test_data_prep.to_csv("forum/sublabels/"+set_name+"_sublabel_preproc.csv")
    test_data_prep.to_csv("forum/valence/"+set_name+"_valence_preproc.csv")

In [3]:
training_data = pd.read_csv("forum/label_train_preproc_drop.csv", index_col="Unnamed: 0")
training_data_proc = preprocessing(training_data)
nan_value = float("NaN")
training_data_proc.replace("", nan_value, inplace=True)
training_data_proc.dropna(how='any', inplace=True)
training_data_proc.head()

,Unnamed: 0.1,Sentence,labels,function_Sentence,nonfunction_Sentence,non_noun_adj,non_noun_verb_adj,non_verb,non_verb_aux,non_noun
Unnamed: 0,,,,,,,,,,
1779,0,"Ich bin mir sicher, ich schaffe es ohne OP ni...",0,Ich bin mir ich es ohne für mich ist es eine,"sicher , schaffe OP nicht , hilfreiche Krücke .","Ich bin mir sicher , ich schaffe es ohne nic...","Ich bin mir sicher , ich es ohne nicht , für...","Ich bin mir sicher , ich es ohne OP nicht , ...","Ich mir sicher , ich es ohne OP nicht , für ...","Ich bin mir sicher , ich schaffe es ohne nic..."
3475,1,Ich Frage mich vor jedem Essen kurz: Brauche ...,1,Ich mich vor jedem ich das,Frage Essen kurz : Brauche jetzt ?,Ich mich vor jedem kurz : Brauche ich das je...,Ich mich vor jedem kurz : ich das jetzt ?,Ich Frage mich vor jedem Essen kurz : ich da...,Ich Frage mich vor jedem Essen kurz : ich da...,Ich mich vor jedem kurz : Brauche ich das je...
3591,2,Ich weiß zwar was was hat und wie viel ich da...,1,Ich was was viel ich zu mir es auf,"weiß zwar hat und wie dann ca nehme , schrei...",Ich weiß zwar was was hat und wie viel ich d...,Ich zwar was was und wie viel ich dann zu mi...,Ich zwar was was und wie viel ich dann ca zu...,Ich zwar was was und wie viel ich dann ca zu...,Ich weiß zwar was was hat und wie viel ich d...
1503,3,"Mein Gewicht steht so rum und ich finde, es k...",0,Mein ich es könnte,"Gewicht steht so rum und finde , noch etwas ...","Mein steht so rum und ich finde , es könnte ...","Mein so rum und ich , es könnte noch etwas m...","Mein Gewicht so rum und ich , es könnte noch...","Mein Gewicht so rum und ich , es noch etwas ...","Mein steht so rum und ich finde , es könnte ..."
496,4,mit dem Entschluss - du musst was dagegen unte...,0,mit dem du was,Entschluss - musst dagegen unternehmen .,mit dem - du musst was dagegen unternehmen .,mit dem - du was dagegen .,mit dem Entschluss - du was dagegen .,mit dem Entschluss - du was dagegen .,mit dem - du musst was dagegen unternehmen .


In [4]:
training_data.to_csv("forum/label_train_preproc_drop.csv")